In [1]:
from ultralytics import settings

settings.update({'datasets_dir': './data/'})

# View all settings
print(settings)

# Return a specific setting
value = settings['runs_dir']

{'settings_version': '0.0.4', 'datasets_dir': './data/', 'weights_dir': '/Users/tuan/Downloads/web_components_classify/weights', 'runs_dir': '/Users/tuan/Downloads/web_components_classify/runs', 'uuid': 'c29d1be8a040bb6ab57fae2b440c98f394dd16742c28b742db5629233856b801', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}


In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Specify the dataset directory where the 'checked', 'unchecked', and 'other' folders are located
dataset_directory = './data/'  # Replace with your dataset directory

# Specify the directories for the train, val, and test sets
train_directory = os.path.join(dataset_directory, 'train')
val_directory = os.path.join(dataset_directory, 'val')
test_directory = os.path.join(dataset_directory, 'test')

# Create the train, val, and test directories if they don't exist
os.makedirs(train_directory, exist_ok=True)
os.makedirs(val_directory, exist_ok=True)
os.makedirs(test_directory, exist_ok=True)

# Split ratios for train, validation, and test
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Ensure the split ratios sum up to 1, using rounding for floating-point precision
assert round(train_ratio + val_ratio + test_ratio, 10) == 1, "Split ratios must sum up to 1"

# Process each class directory ('checked', 'unchecked', 'other')
for class_name in ['checked', 'unchecked', 'other']:
    # Path to the class directory
    class_dir = os.path.join(dataset_directory, class_name)
    
    # Get a list of all files in the class directory
    all_files = [file for file in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, file))]

    # First split to separate out the test set
    initial_train_files, test_files = train_test_split(all_files, test_size=test_ratio, random_state=42)
    
    # Adjust the train ratio to account for the removed test set
    adjusted_train_ratio = train_ratio / (1 - test_ratio)
    
    # Second split to separate the remaining files into train and val sets
    train_files, val_files = train_test_split(initial_train_files, train_size=adjusted_train_ratio, random_state=42)

    # Function to copy the files to the respective train, val, or test directories
    def copy_files(files, dest_dir):
        os.makedirs(os.path.join(dest_dir, class_name), exist_ok=True)
        for file in files:
            src_path = os.path.join(class_dir, file)
            dest_path = os.path.join(dest_dir, class_name, file)
            shutil.copy(src_path, dest_path)

    # Copy files to the respective directories
    copy_files(train_files, train_directory)
    copy_files(val_files, val_directory)
    copy_files(test_files, test_directory)

print("Done splitting into train, val, and test sets.")


Done splitting into train, val, and test sets.


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n-cls.pt')
model.train(data='./data/', epochs=500, imgsz=64)